#### Model is overfitting since it is giving us 100 f1_score but since it also gives good test prediction so we are kepping it for now

In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score, \
                            confusion_matrix, plot_confusion_matrix, precision_recall_curve, plot_roc_curve, \
                            roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [4]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(oob_score=True, n_estimators=100, class_weight="balanced")

In [5]:
from sklearn.decomposition import PCA

In [6]:
def classifier_report(y_true, y_pred):
    
    print("Precision :  ", precision_score(y_true, y_pred))
    print("Recall    :  ", recall_score(y_true, y_pred))
    print("F1-score  :  ", f1_score(y_true, y_pred))
    print("Accuracy  :  ", accuracy_score(y_true, y_pred))
    print("ROC AUC   :  ", roc_auc_score(y_true, y_pred))
    print("Confusion Matrix : ")
    print(confusion_matrix(y_true, y_pred))

In [7]:
data = pd.read_csv("../preprocessed_data.csv")
test_data = pd.read_csv("../preprocessed_test_data.csv").drop(columns=["id"])

In [8]:
data.drop(columns=["topic_4", "topic_5"], inplace=True)
test_data.drop(columns=["topic_4", "topic_5"], inplace=True)

data.columns

Index(['tweet', 'label', 'word_count', 'charcter_count', 'hashtag_count',
       'vulgar_word_count', 'symbol_count', 'stop_word_count',
       'sentiment_score', 'topic_1', 'topic_2', 'topic_3'],
      dtype='object')

In [9]:
# def clean(tweet):
    
#     new_tweet = ""
#     for word in tweet.split():
#         if word not in most_common:
#             new_tweet += word
#             new_tweet += " "
        
#     return new_tweet

# most_common = []
# for word in Counter("".join(data["tweet"]).split()).most_common()[:100]:
#     score = sid.polarity_scores(word[0])["compound"]
#     if (score > -0.2 and score < 0.2) :
#         most_common.append(word[0])

# data["tweet"] = data["tweet"].apply(clean)
# test_data["tweet"] = test_data["tweet"].apply(clean)

In [10]:
# for word in Counter("".join(data["tweet"]).split()).most_common()[:100]:
#         score = sid.polarity_scores(word[0])["compound"]
#         if (score > -0.2 and score < 0.2) :
#             print(word[0], " : ", score)

In [11]:
# tfidf_vector = tfidf_vectorizer.fit_transform(data["tweet"])
# test_tfidf_vector = tfidf_vectorizer.transform(test_data["tweet"])

In [12]:
# pca = PCA(n_components=1)
# columns = []
# for i in range(pca.n_components):
#     columns.append("feature " + str(i))
    
# pca_transformed = pd.DataFrame(pca.fit_transform(tfidf_vector.todense()), columns=columns)
# test_pca_transformed = pd.DataFrame(pca.transform(test_tfidf_vector.todense()), columns=columns)

# for feature in columns:
#     data[feature] = pca_transformed[feature]
#     test_data[feature] = test_pca_transformed[feature]

In [13]:
random_forest = RandomForestClassifier(n_estimators=50, max_depth=6)
np.array(cross_val_score(random_forest, data.drop(columns=["tweet", "label"]), data["label"], cv=10)).mean()
# 0.891919191919192

0.8878787878787879

In [14]:
random_forest.fit(data.drop(columns=["tweet", "label"]), data["label"])

RandomForestClassifier(max_depth=6, n_estimators=50)

In [15]:
prediction = pd.DataFrame({"label": random_forest.predict(test_data.drop(columns=["tweet"]))})
prediction.to_csv("random_forest_prediction.csv")

In [16]:
pickle.dump(random_forest, open("random_forest.sav", "wb"))

In [17]:
prediction["label"].value_counts(normalize=True)

0    0.72043
1    0.27957
Name: label, dtype: float64

In [18]:
# 0    0.716846
# 1    0.283154